In [ ]:
#Download videos from youtube, form the list of downloader videos to video_id_train.txt
import os
import json
import pafy

# specify download directory
directory = '/home/yulia/github/Project_DVCVQA_ver1/helping/download_videos_train_full/'
resFile_video = '/home/yulia/github/Activity_net_whole/train_qa.json'
ques_video = '/home/yulia/github/Activity_net_whole/activitynet-qa/dataset/train_q10.json'
quesFile = '/home/yulia/github/Activity_net_whole/train10_qa_questions.json'
data = json.load(open(resFile_video))
vid_to_ques_id = data['question']

ques_data = json.load(open(quesFile))
ques_info_data = json.load(open(ques_video))


videoCounter = 0

# these lists will record unavailable videos by their key so we can find them 
# in the downloaded files. it's important to remove these fake files from 
# datset files for proper training.
unavailable_tr = []
unavailable_tst = []
unavailable_val = []

# open json file
with open('/home/yulia/github/Activity_net_whole/ActivityNet/activity_net.v1-3.min.json') as data_file:    
    data = json.load(data_file)

videos = data['database']
video_id_train = open("video_id_train_full.txt","r") 
# take only video informations from database object
not_downloaded = open("not_downloaded.txt","a") 

# print(len(vid_to_ques_id.keys()))

for key in vid_to_ques_id.keys():
    
# iterate through dictionary of videos
# for key in videos:
    # take video
    video_id = key[2:]
    video_data = videos[video_id]
    
#     # find video subset
    subset = video_data['subset']

#     # find video label
    annotations = video_data['annotations']
#     print(video_id)
    
    label = ''
    if len(annotations) != 0:
        label = annotations[0]['label']
        label = '/' + label.replace(' ', '_')

#     # create folder named as <label> if does not exist
#     label_dir = directory + subset + label
#     if not os.path.exists(label_dir):
#         os.makedirs(label_dir)

    # take url of video
    url = video_data['url']
    
    try:
        # start to download
        video = pafy.new(url)
        best = video.getbest()     
        filename = best.download(filepath=directory + '/' + video_id + '.avi')
        print('Downloading... ' + str(videoCounter) + '\n')
        videoCounter += 1
        
    except Exception as inst:
        not_downloaded.write(key + "\n") 
        
        if subset == 'training':
            unavailable_tr.append(key)
            print('Number of Unavailable Training Videos: ',len(unavailable_tr))
            print('URL: ', url)
            
        if subset == 'validation':
            unavailable_val.append(key)
            print('Number of Unavailable Validation Videos: ', len(unavailable_val))
            print('URL: ', url)
            
        if subset == 'testing':
            unavailable_tst.append(key)
            print('Number of Unavailable Testing Videos: ', len(unavailable_tst))
            print('URL: ', url)
            
video_id_train.close()
not_downloaded.close()        
# overall unavailable videos
print('Number of Unavailable training videos: ',len(unavailable_tr))
print('Number of Unavailable validation videos: ',len(unavailable_val))
print('Number of Unavailable testing videos: ',len(unavailable_tst))

In [ ]:
import json
import time
import os
import operator
import numpy as np
import h5py
import torch

from collections import defaultdict
from torch.utils.data import DataLoader, Dataset

resFile_video = '/home/yulia/github/Activity_net_whole/train_qa.json'
quesFile = '/home/yulia/github/Activity_net_whole/train10_qa_questions.json'
# ques_video = '/home/yulia/github/Activity_net_whole/train_q10.json'
ques_video = '/home/yulia/github/Activity_net_whole/activitynet-qa/dataset/train_q10.json'

data = json.load(open(resFile_video))
ques_data = json.load(open(quesFile))
ques_info_data = json.load(open(ques_video))

vid_to_ques_id = data['question']
question_id_dict = data['question_id']
answer_dict = data['answer_info']
question_info = data['question_info']
caption_info = data['caption_info']

video_id_train = open("video_id_train_full.txt","a") 
for video_id in caption_info.keys():
    video_id_train.write(video_id + "\n") 

video_id_train.close()